# The JuMP ecosystem for mathematical optimization: MOI

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [1]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

INFO: Recompiling stale cache file /Users/jvielma/.julia/lib/v0.6/JuMP.ji for module JuMP.


## MOI and JuMP

In [64]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

Behind every JuMP model there is a MOI model (used to be a MPB model)

In [6]:
typeof(model.moibackend)

MathOptInterface.Bridges.LazyBridgeOptimizer{MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMP.JuMPMOIModel{Float64}}},MathOptInterface.Bridges.AllBridgedConstraints{Float64}}

### Recovering variables:

In [33]:
xx = MOI.get(model.moibackend,MOI.ListOfVariableIndices()) 

2-element Array{MathOptInterface.VariableIndex,1}:
 MathOptInterface.VariableIndex(1)
 MathOptInterface.VariableIndex(2)

In [39]:
x[1].index == xx[1]

true

In [42]:
x[1].m == model

true

In [84]:
typeof(x[1])

JuMP.VariableRef

JuMP variable : MOI variable + owning model
```julia
struct VariableRef 
    m::Model
    index::MOI.VariableIndex
end
```

MOI variable : Type-safe unique and not necessarily consecutive index
```julia
struct VariableIndex
    value::Int64
end
```

### Recovering constraints:

In [45]:
L,Q,C = MOI.get(model.moibackend,MOI.ListOfConstraints())

3-element Array{Tuple{DataType,DataType},1}:
 (MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64})   
 (MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64})
 (MathOptInterface.VectorAffineFunction{Float64}, MathOptInterface.SecondOrderCone)     

MOI constraints are of the form 
$$
f(x) \in S
$$
were
* $f$ is an ``MOI.AbstractFunction`` (e.g. ``MOI.ScalarAffineFunction`` = $a^T x +b$ for some $a,b$) and
* $S$ is an ``MOI.AbstractSet`` (e.g. ``MOI.LessThan`` = $\{y\,:\, y\leq u\}$ for some $u$)

#### Linear inequalities (``MOI.ScalarAffineFunction``-in-``MOI.LessThan``):

In [53]:
linear = MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{L...}())

2-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(1)
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(2)

In [67]:
linear1.index == linear[1]

true

In [68]:
fieldnames(linear1)

2-element Array{Symbol,1}:
 :m    
 :index

In [69]:
linear1.m == model

true

In [85]:
typeof(linear1)

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}

JuMP constraint : MOI constraint + owning model
```julia
struct ConstraintRef{C}
    m::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end
```

MOI constraint : Type-safe wrapper of index to constraint of class F-in-S. Index not necessarily consecutive and unique within constraint class
```julia
struct ConstraintIndex{F,S}
    value::Int64
end
```

Linear inequalities 1 : 
* ``2*x[1] + x[2] <= 1``, or
* $a^T x + b \in \{y:y\leq u\}$ for $a=(2,1)^T$, $b=0$ and $u=0$ 

$a=(2,1)^T$:

In [56]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).terms

2-element Array{MathOptInterface.ScalarAffineTerm{Float64},1}:
 MathOptInterface.ScalarAffineTerm{Float64}(2.0, MathOptInterface.VariableIndex(1))
 MathOptInterface.ScalarAffineTerm{Float64}(1.0, MathOptInterface.VariableIndex(2))

$b=0$:

In [58]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).constant

0.0

$u=0$:

In [60]:
MOI.get(model.moibackend,MOI.ConstraintSet(),linear[1]).upper

1.0

Similar for othet constrsints:

In [49]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarQuadraticFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{Q...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}}(3)

In [51]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.VectorAffineFunction{Float64}, MOI.SecondOrderCone}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{C...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(4)

## MOI Constraints

| Mathematical Constraint       | MOI Function                 | MOI Set        |
|-------------------------------|------------------------------|----------------|
| $a^Tx = b$                  | `ScalarAffineFunction`       | `EqualTo`      |
| $l \le a^Tx \le u$          | `ScalarAffineFunction`       | `Interval`     |
| $x_i \le u$                 | `SingleVariable`             | `LessThan`     |
| $Ax + b \in \mathbb{R}_+^n$ | `VectorAffineFunction`       | `Nonnegatives` |
| $\lVert Ax + b\rVert_2 \le c^Tx + d$                        | `VectorAffineFunction`       | `SecondOrderCone` |
| $y \exp (x/y) \le z, y > 0$  | `VectorOfVariables`       | `ExponentialCone`                  |
| $x \in \mathbb{R}^{d\left(d+1\right)/2}$,◹$(x)\in \text{PSD} \subseteq \mathbb{R}^{d\times d}$                                       | `VectorOfVariables`          | `PositiveSemidefiniteConeTriangle` |
| $x^TQx + a^Tx + b \ge 0$    | `ScalarQuadraticFunction`    | `GreaterThan`                 |
| $x_i \in \mathbb{Z}$                                                                     | `SingleVariable`    | `Integer`                          |
| $x_i \in \{0,1\}$                                                                        | `SingleVariable`    | `ZeroOne`                          |
| $x_i \in \{0\} \cup [l,u]$                                                               | `SingleVariable`    | `Semicontinuous`                   |
| At most one component of $x$ can be nonzero                                              | `VectorOfVariables` | `SOS1`                             |

MOI includes standard functions and sets, and allows for some extensions. See [MOI manual](http://www.juliaopt.org/MathOptInterface.jl/stable/index.html) for more details on [supported sets and functions](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Standard-form-problem-1) and more [constraint examples](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Constraints-by-function-set-pairs-1).

### Note: there are multiple ways to write the same constraint. For more details and discussions see [Constraint Bridges](#Constraint-Bridges).

## A Complete MOI Model

In [3]:
using MathOptInterface
const MOI = MathOptInterface
using GLPK

# Solve the binary-constrained knapsack problem: max c'x: w'x <= C, x binary using GLPK.

c = [1.0, 2.0, 3.0]
w = [0.3, 0.5, 1.0]
C = 3.2

numvariables = length(c)

optimizer = GLPK.GLPKOptimizerMIP()

# create the variables in the problem
x = MOI.addvariables!(optimizer, numvariables)

# set the objective function
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set!(optimizer, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(), objective_function)
MOI.set!(optimizer, MOI.ObjectiveSense(), MOI.MaxSense)

# add the knapsack constraint
knapsack_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0)
MOI.addconstraint!(optimizer, knapsack_function, MOI.LessThan(C))

# add integrality constraints
for i in 1:numvariables
    MOI.addconstraint!(optimizer, MOI.SingleVariable(x[i]), MOI.ZeroOne())
end

# all set
MOI.optimize!(optimizer)

termination_status = MOI.get(optimizer, MOI.TerminationStatus())
objvalue = MOI.canget(optimizer, MOI.ObjectiveValue()) ? MOI.get(optimizer, MOI.ObjectiveValue()) : NaN
if termination_status != MOI.Success
    error("Solver terminated with status $termination_status")
end

@assert MOI.get(optimizer, MOI.ResultCount()) > 0

result_status = MOI.get(optimizer, MOI.PrimalStatus())
if result_status != MOI.FeasiblePoint
    error("Solver ran successfully did not return a feasible point. The problem may be infeasible.")
end
primal_variable_result = MOI.get(optimizer, MOI.VariablePrimal(), x)

@show objvalue
@show primal_variable_result

objvalue = 6.0
primal_variable_result = 

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

[1.0, 1.0, 1.0]


## Model modifications

In [44]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, linear2, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, quadratic,x[1]^2 + x[2]^2 <= 1)  
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

delete variables or constraints

In [45]:
if MOI.candelete(model.moibackend,x[1].index)
    MOI.delete!(model.moibackend,x[1].index)
end
if MOI.candelete(model.moibackend,linear1.index)
    MOI.delete!(model.moibackend,linear1.index)
end

modify constraints

In [47]:
if MOI.canmodify(model.moibackend, typeof(linear2.index), MOI.ScalarCoefficientChange{Float64})
    MOI.modify!(model.moibackend, linear2.index, MOI.ScalarCoefficientChange(x[1].index, 3.0))
end

or completely transform them 

In [48]:
rsocp = MOI.transform!(model.moibackend, socp.index, MOI.RotatedSecondOrderCone(3))

MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.RotatedSecondOrderCone}(5)

In [51]:
rsocp != socp

true

In [55]:
MOI.isvalid(model.moibackend,socp.index)

false

## MOI Attributes

In [86]:
model = GLPK.GLPKOptimizerMIP()
var_index = MOI.addvariable!(model)
constr_index = MOI.addconstraint!(model,MOI.SingleVariable(var_index),MOI.LessThan(10.0))

MathOptInterface.ConstraintIndex{MathOptInterface.SingleVariable,MathOptInterface.LessThan{Float64}}(1)

for building models...

In [95]:
MOI.set!(model,MOI.ObjectiveSense(),MOI.MaxSense)
MOI.set!(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), MOI.SingleVariable(var_index))

0

and more...

In [113]:
if MOI.canset(model,MOI.VariablePrimalStart(),typeof(var_index))
    MOI.set!(model,MOI.VariablePrimalStart(),var_index,5.0)
end
if MOI.canset(model,MOI.ConstraintDualStart(),typeof(constr_index))
    MOI.set!(model,MOI.ConstraintDualStart(),constr_index,5.0)
end

### Also for JuMP models

In [121]:
model = Model()

A JuMP Model

some attributes have custom macro syntax

In [122]:
@variable(model, x >= 10.0, start = 5.0)
@objective(model, Max, x)

MathOptInterface.SingleVariable(MathOptInterface.VariableIndex(1))

others use MOI functions

In [126]:
bound_ref = JuMP.LowerBoundRef(x).index
if MOI.canset(model.moibackend,MOI.ConstraintDualStart(),typeof(bound_ref))
    MOI.set!(model.moibackend,MOI.ConstraintDualStart(),bound_ref,10.0)
end

10.0

## Constraint Bridges

Missing bridges https://github.com/JuliaOpt/MathOptInterface.jl/issues/423

unsupported constraints https://github.com/JuliaOpt/JuMP.jl/pull/1386